In [ ]:
# 필요한 모듈 설치
!pip3 install -r ./requirement.txt

In [7]:
# 모듈 로드
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

In [2]:
# FAISS Class 구현
class FAISSNearestNeighbor:
    def __init__(self, vectors, texts):
        self.vectors = np.array(vectors).astype('float32')
        self.texts = texts
        self.d = self.vectors.shape[1] # 벡터 차원
        # faiss.normalize_L2(self.vectors) # 정규화는 SBERT에서 이미 하였으므로 생략
        self.index = faiss.IndexFlatIP(self.d)
        self.index.add(self.vectors)
    
    def search(self, query_vector, top_k=1):
        query_vector = np.array(query_vector).astype('float32').reshape(1, -1) # 입력 벡터는 1개이므로 reshape 필요
        # faiss.normalize_L2(self.vectors) # 정규화는 SBERT에서 이미 하였으므로 생략
        similarities, indices = self.index.search(query_vector, top_k)
        similar_texts = [self.texts[idx] for idx in indices[0]]
        return similar_texts, similarities[0]

In [3]:
# 테스트 코드 - VectorDB 구축
model = SentenceTransformer("jhgan/ko-sbert-sts")

text_list = [
    "원숭이가 노래를 한다.",
    "배가 바다를 떠나 원대한 여정을 시작했다.",
    "그는 자신이 벌레만도 못한 취급을 받을 것이라곤 생각할 수 없었다."
]
text_vectors = model.encode(text_list, normalize_embeddings=True)
vector_db = FAISSNearestNeighbor(text_vectors, text_list)

In [ ]:
# 테스트 코드 - VectorDB 테스트
query_text = "그배가 바다를 떠나 원대한 여정을 시작함."

text, sim = vector_db.search(model.encode(query_text, normalize_embeddings=True))
print(f"유사 문장 : {text[0]}\t유사도 : {sim[0] : .4f}")
print(f"SBERT 유사도 : {float(util.cos_sim(model.encode(text), model.encode(query_text))) : .4f}") # FAISS 유사도와 동일해야함!

유사 문장 : 배가 바다를 떠나 원대한 여정을 시작했다.	유사도 :  0.9299
SBERT 유사도 :  0.9299
